In [1]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
pip install -U deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


In [6]:
from deep_translator import (GoogleTranslator)

In [7]:
langs_dict = GoogleTranslator().get_supported_languages(as_dict=True)  # output: {arabic: ar, french: fr, english:en etc...}
print(langs_dict)

{'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'j

In [8]:
lang = "hi"
translator = GoogleTranslator(source='en', target=lang)

# Data Import

The training data will have a real-values semantic textual relatedness score (between 0 and 1) for a pair of Marathi-language sentences.

The data is structured as a CSV file with the following fields:
- PairID: a unique identifier for the sentence pair
- Text: two sentences separated by a newline ('\n') character
- Score: the semantic textual relatedness score for the two sentences

Below we will show you how to load and re-format the provided data file.

In [2]:
# Load the File
df_str_rel = pd.read_csv('https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20A/eng/eng_train.csv')
df_str_rel.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [3]:
y = df_str_rel['Score']

In [4]:
# Creating a column "Split_Text" which is a list of two sentences.
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

,PairID,Text,Score,Split_Text
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"[It that happens, just pull the plug., if that..."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,"[A black dog running through water., A black d..."
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,"[I've been searchingthe entire abbey for you.,..."
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,[If he is good looking and has a good personal...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"[She does not hate you, she is just annoyed wi..."


In [10]:
# from tensorflow.keras.preprocessing.text import Tokenizer

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df_str_rel['Split_Text'].apply(lambda x: x[0] + ' ' + x[1]))

# Convert each text to sequences
X1 = df_str_rel['Split_Text'].apply(lambda x: x[0])
X2 = df_str_rel['Split_Text'].apply(lambda x: x[1])

type(X1)

pandas.core.series.Series

In [11]:
X1 = X1.values
X2 = X2.values
type(X1)

numpy.ndarray

In [12]:
df_str_rel = df_str_rel.assign(sentence1 = X1)
df_str_rel = df_str_rel.assign(sentence2 = X2)
df_str_rel

,PairID,Text,Score,Split_Text,sentence1,sentence2
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"[It that happens, just pull the plug., if that...","It that happens, just pull the plug.","if that ever happens, just pull the plug."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,"[A black dog running through water., A black d...",A black dog running through water.,A black dog is running through some water.
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,"[I've been searchingthe entire abbey for you.,...",I've been searchingthe entire abbey for you.,I'm looking for you all over the abbey.
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,[If he is good looking and has a good personal...,If he is good looking and has a good personali...,"If he's good looking, and a good personality, ..."
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"[She does not hate you, she is just annoyed wi...","She does not hate you, she is just annoyed wit...","She doesn't hate you, she is just annoyed."
...,...,...,...,...,...,...
5495,ENG-train-5495,A young boy pounding on an anvil.\nWoman sits ...,0.0,"[A young boy pounding on an anvil., Woman sits...",A young boy pounding on an anvil.,Woman sits on the curb talking on a cellphone.
5496,ENG-train-5496,I love how he recognized his wife tempered his...,0.0,[I love how he recognized his wife tempered hi...,I love how he recognized his wife tempered his...,"Torpedo Ink is Viktor's Band of Brothers, the ..."
5497,ENG-train-5497,I actually read a chapter or two beyond that p...,0.0,[I actually read a chapter or two beyond that ...,I actually read a chapter or two beyond that p...,Lets say she's a blend of two types of beings.
5498,ENG-train-5498,A boy gives being in the snow two thumbs up.\n...,0.0,"[A boy gives being in the snow two thumbs up.,...",A boy gives being in the snow two thumbs up.,A satisfied cat is perched beside a crystal lamp.


In [13]:
df_str_rel.drop(columns = ['Text', 'Split_Text'], inplace = True)
df_str_rel

,PairID,Score,sentence1,sentence2
0,ENG-train-0000,1.0,"It that happens, just pull the plug.","if that ever happens, just pull the plug."
1,ENG-train-0001,1.0,A black dog running through water.,A black dog is running through some water.
2,ENG-train-0002,1.0,I've been searchingthe entire abbey for you.,I'm looking for you all over the abbey.
3,ENG-train-0003,1.0,If he is good looking and has a good personali...,"If he's good looking, and a good personality, ..."
4,ENG-train-0004,1.0,"She does not hate you, she is just annoyed wit...","She doesn't hate you, she is just annoyed."
...,...,...,...,...
5495,ENG-train-5495,0.0,A young boy pounding on an anvil.,Woman sits on the curb talking on a cellphone.
5496,ENG-train-5496,0.0,I love how he recognized his wife tempered his...,"Torpedo Ink is Viktor's Band of Brothers, the ..."
5497,ENG-train-5497,0.0,I actually read a chapter or two beyond that p...,Lets say she's a blend of two types of beings.
5498,ENG-train-5498,0.0,A boy gives being in the snow two thumbs up.,A satisfied cat is perched beside a crystal lamp.


In [15]:


df_str_rel['sentence1'] = df_str_rel['sentence1'].apply(lambda x: translator.translate(x))
df_str_rel['sentence2'] = df_str_rel['sentence2'].apply(lambda x: translator.translate(x))


In [16]:
df_str_rel = df_str_rel[['PairID', 'sentence1', 'sentence2', 'Score']]
df_str_rel

,PairID,sentence1,sentence2,Score
0,ENG-train-0000,"ऐसा होता है, बस प्लग खींचें।","यदि ऐसा कभी होता है, तो बस प्लग खींच लें।",1.0
1,ENG-train-0001,एक काला कुत्ता पानी में दौड़ रहा है.,एक काला कुत्ता पानी के बीच से भाग रहा है।,1.0
2,ENG-train-0002,मैं तुम्हारे लिए पूरे मठ की खोज कर रहा हूं।,मैं पूरे मठ में तुम्हें ढूंढ रहा हूं।,1.0
3,ENG-train-0003,यदि वह दिखने में अच्छा है और उसका व्यक्तित्व अ...,यदि वह दिखने में अच्छा है और उसका व्यक्तित्व अ...,1.0
4,ENG-train-0004,"वह तुमसे नफरत नहीं करती, बस तुमसे नाराज़ है।","वह तुमसे नफरत नहीं करती, वह बस नाराज़ है।",1.0
...,...,...,...,...
5495,ENG-train-5495,एक युवा लड़का निहाई पर जोर से पैर मार रहा है।,महिला सड़क पर बैठकर सेलफोन पर बात कर रही है।,0.0
5496,ENG-train-5496,मुझे अच्छा लगा कि कैसे उसने पहचाना कि उसकी पत्...,"टॉरपीडो इंक विक्टर के ब्रदर्स बैंड हैं, जो एकम...",0.0
5497,ENG-train-5497,मैंने वास्तव में उस बिंदु से आगे एक या दो अध्य...,मान लीजिए कि वह दो प्रकार के प्राणियों का मिश्...,0.0
5498,ENG-train-5498,एक लड़का बर्फ़ में होने पर दो अंगूठे ऊपर उठाता...,एक संतुष्ट बिल्ली एक क्रिस्टल लैंप के पास बैठी...,0.0


# Preprocessing is done till here.

In [17]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fd71d84a40a2b3f6cc4b5d77120148a00e24c72e5fb0803db8858d27c72c140b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [18]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample
from sentence_transformers import losses

model = SentenceTransformer( "l3cube-pune/hindi-sentence-similarity-sbert" )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [19]:
ds = df_str_rel[ [ 'sentence1', 'sentence2', 'Score'] ].to_numpy()

train_examples = []
for i in range( ds.shape[0] ):
  train_examples.append( InputExample(texts=[ ds[i][0] , ds[i][1] ] , label=ds[i][2] ) )


In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [21]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/344 [00:00<?, ?it/s]

# Training completed here

In [22]:
test_ds = pd.read_csv( "https://raw.githubusercontent.com/semantic-textual-relatedness/Semantic_Relatedness_SemEval2024/main/Track%20C/hin/hin_dev.csv" )
test_ds

,PairID,Text
0,HIN-dev-00001,"""जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को ..."
1,HIN-dev-00002,"""राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने ..."
2,HIN-dev-00003,"""बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वरु..."
3,HIN-dev-00004,"""घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र की..."
4,HIN-dev-00005,"""शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियों..."
...,...,...
283,HIN-dev-00284,"""' तसलीमा नसरीन ने इस दौरान यह भी कहा कि इस्ला..."
284,HIN-dev-00285,"""देश में हर साल 10 फरवरी को कृमि दिवस मनाया जा..."
285,HIN-dev-00286,"""पाक के सरकारी प्रवक्ता का ट्विटर खाता बंद,यहा..."
286,HIN-dev-00287,"""छोटे पर्दे पर भी वह काफी ज्यादा छाए रहे।""\n""स..."


In [23]:
test_ds['Split_Text'] = test_ds['Text'].apply(lambda x: x.split("\n"))
test_ds.head()

,PairID,Text,Split_Text
0,HIN-dev-00001,"""जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को ...","[""जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को..."
1,HIN-dev-00002,"""राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने ...","[""राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने..."
2,HIN-dev-00003,"""बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वरु...","[""बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वर..."
3,HIN-dev-00004,"""घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र की...","[""घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र क..."
4,HIN-dev-00005,"""शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियों...","[""शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियो..."


In [24]:
X1_dev = test_ds['Split_Text'].apply(lambda x: x[0])
X2_dev = test_ds['Split_Text'].apply(lambda x: x[1])

test_ds = test_ds.assign(sentence1 =X1_dev)
test_ds = test_ds.assign(sentence2 =X2_dev)

test_ds.drop(columns = ['Text', 'Split_Text'], inplace = True)

test_ds

,PairID,sentence1,sentence2
0,HIN-dev-00001,"""जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को ...","""आरोपी एसीबी अधिकारी बनकर लोगो से चौथ वसूली कर..."
1,HIN-dev-00002,"""राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने ...","""राज्यपाल उपलब्ध नहीं थे, इसलिए उपराज्यपाल ने ..."
2,HIN-dev-00003,"""बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वरु...","""अभिनेता वरुण धवन ने कहा है कि अभिनेता-पहलवान ..."
3,HIN-dev-00004,"""घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र की...","""प्राप्त जानकारी के अनुसार घटना डरबन में घटित ..."
4,HIN-dev-00005,"""शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियों...","""पुलिस ने मामला दर्ज कर लिया है, लेकिन अभी तक ..."
...,...,...,...
283,HIN-dev-00284,"""' तसलीमा नसरीन ने इस दौरान यह भी कहा कि इस्ला...","""इनमें भाग लेने वाला इस्लाम और कश्मीर का दुश्म..."
284,HIN-dev-00285,"""देश में हर साल 10 फरवरी को कृमि दिवस मनाया जा...","""मैथ्यू हेडन के नाम है सबसे बडा स्कोर, देखें ट..."
285,HIN-dev-00286,"""पाक के सरकारी प्रवक्ता का ट्विटर खाता बंद,यहा...","""माइक्रो-ब्लॉगिंग साइट ट्विटर ने पाकिस्तान के ..."
286,HIN-dev-00287,"""छोटे पर्दे पर भी वह काफी ज्यादा छाए रहे।""","""साल 2016 में गूगल पर सबसे ज्यादा सर्च हुए टाॅ..."


In [25]:
def cosine( e1 , e2 ):
  return np.dot( e1.T , e2 ) / ( np.linalg.norm( e1 , ord=2 ) * np.linalg.norm( e2 , ord=2 ) )

test_ds = test_ds.to_numpy()
scores = []
for id , s1 , s2 in test_ds:
  e = model.encode( [ s1 , s2 ] )
  scores.append( cosine( e[0] , e[1] ) )


# Generate file for submission

Submission file has two columns: '**PairID**' and '**Pred_Score**'

In [26]:
pred_ds = {
    "PairID": test_ds[ : , 0 ] ,
    "Pred_Score": scores
}
pred_ds = pd.DataFrame.from_dict( pred_ds )
pred_ds

,PairID,Pred_Score
0,HIN-dev-00001,0.413189
1,HIN-dev-00002,0.879455
2,HIN-dev-00003,0.465663
3,HIN-dev-00004,0.354235
4,HIN-dev-00005,0.444987
...,...,...
283,HIN-dev-00284,0.290161
284,HIN-dev-00285,0.170811
285,HIN-dev-00286,0.683678
286,HIN-dev-00287,0.190906


In [27]:
pred_ds.to_csv( "pred_hin_c.csv", index = False )